# Generating Active case data from https://coronavirus.data.gov.uk/

In [ ]:
import sys
from pathlib import Path
#Set root to be the main project folder
#Note that this notebook is in /SOCIAL_DISTANCING/CODE/NOTEBOOKS/TEST_NOTEBOOKS
root = Path.cwd().parent.parent
print(root)
py_path = Path(root/'code/py-files')
print(py_path)
data_path = Path(root/'static')
print(data_path)

#Add location of py files to path so we can import
sys.path.insert(0,str(py_path))

In [272]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from importlib import reload
from tqdm import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm
from scipy.spatial import distance_matrix

In [273]:
utla_folder = Path(data_path/'utla_data/')

In [274]:
import os
import requests

url = "https://api.coronavirus.data.gov.uk/v2/data?areaType=utla&metric=cumCasesBySpecimenDate&metric=newCasesBySpecimenDate&format=csv"
req = requests.get(url)
url_content = req.content
csv_file = open(utla_folder/'csv/covid_cases.csv', 'wb')

csv_file.write(url_content)
csv_file.close()

In [275]:
df_covid = pd.read_csv(utla_folder/'csv/covid_cases.csv')

In [333]:
# no data for City of London E09000001
df_covid[df_covid.areaCode == "E09000001"]

,index,areaCode,areaName,areaType,date,cumCasesBySpecimenDate,newCasesBySpecimenDate


In [277]:
df_pops = pd.read_csv(utla_folder/'csv/population.csv')
df_pops = df_pops.iloc[:151]

In [278]:
utlasA = df_covid["areaCode"].unique()
utlasA.sort()

In [279]:
gdf_ut = gpd.read_file(utla_folder/"shp/UTLA_ENG.shp")

In [280]:
df_covid = df_covid.sort_values("date").reset_index()

In [281]:
dates = df_covid["date"].unique()
dates.sort()

In [282]:
def GenerCases(date):
    #print(date)
    df_covid_date = df_covid[df_covid["date"] == date].copy()
    df_covid_date = df_covid_date.reset_index(drop=True)
    #print(date)
    TotalCases = np.full(gdf_ut.UTLA19CD.size, 0)
    for j, itemA in enumerate(gdf_ut.UTLA19CD):
        for k, itemB in enumerate(df_covid_date["areaCode"]):
            if itemA == itemB:
                TotalCases[j] =  df_covid_date["cumCasesBySpecimenDate"][k]
    return TotalCases

In [283]:
july_date = np.where(dates == '2020-07-01')[0][0]
dates_from_july = dates[july_date:]

In [284]:
avg_infection_time = 10

In [285]:
cases = dates_from_july[::avg_infection_time]

In [304]:
def produce_cases(cases):
    all_new_cases = []
    old_item =  GenerCases(cases[0])
    for item in cases[1:]:
        new_cases = GenerCases(item) - old_item
        all_new_cases.append(new_cases)
        old_item = GenerCases(item)
    return np.array(all_new_cases)

In [318]:
new_cases = produce_cases(cases)

In [320]:
# Uncomment if you wish to have cases per 1000 people
old_item = cases[0]
for k, item in enumerate(new_cases):
    gdf_ut['{}'.format(cases[k])] =  item #/ df_pops["population"] * 1000
    old_item = cases[k]

In [312]:
#  gdf_ut.to_file(utla_folder/"shp/covid-cases_EN_recent.shp")